# Combinação De Modelos

## Técnicas de ensemble

### Bagging - Bootstrap Aggregating:

É uma amostragem com reposição. Exemplo: Temos um dataframe com 4 linhas. Nós vamos gerar uma amostragem de 4 linhas com reposição, se o dataframe tiver as 4 linhas como 1, 2, 3 e 4, é possivel que a amostragem venha com as 4 linhas como 1, 4, 3, 4, ja que a amostragem é feita de forma aleatória e com a reposição de linhas.  

Amostragem com reposição = Bootstrap  

### Base learners:

Depois de criar as amostragens na quantidade que for preciso, se cria um modelo para cada uma delas. Esses são os Base learners.  

### Aggregating:

No fim é visto o resultado de cada modelo, e é feito uma agregação para adquirir a resposta final.  
A agregação é um metodo que junta todos os resultados de forma que as respostas sao 0 ou 1 e faz uma comparação de frequencia. O resultado com maior frequencia é o resultado final.  

Caso seja um modelo de regressão. É somado o resultado dos modelos e tirado a média. Por exemplo: Temos um banco de dados com dados de imoveis, e precisamos predizer o preço do aluguel. Tiramos tres amostragens, para cada uma fizemos um modelo de regressão. Um modelo resultou em 2500, outro em 2250 e o outro 2750 fazemos a soma dentre esses valores e dividimos pelo numero de modelos resultando em 2500.

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [34]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [36]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [38]:
titanic = titanic.drop(columns=['deck', 'alive'])

In [40]:
titanic = titanic.dropna()
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     712 non-null    int64   
 1   pclass       712 non-null    int64   
 2   sex          712 non-null    object  
 3   age          712 non-null    float64 
 4   sibsp        712 non-null    int64   
 5   parch        712 non-null    int64   
 6   fare         712 non-null    float64 
 7   embarked     712 non-null    object  
 8   class        712 non-null    category
 9   who          712 non-null    object  
 10  adult_male   712 non-null    bool    
 11  embark_town  712 non-null    object  
 12  alone        712 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.4+ KB


In [42]:
titanic = pd.get_dummies(titanic, drop_first=True).astype(int)
titanic.head()

,survived,pclass,age,sibsp,parch,fare,adult_male,alone,sex_male,embarked_Q,embarked_S,class_Second,class_Third,who_man,who_woman,embark_town_Queenstown,embark_town_Southampton
0,0,3,22,1,0,7,1,0,1,0,1,0,1,1,0,0,1
1,1,1,38,1,0,71,0,0,0,0,0,0,0,0,1,0,0
2,1,3,26,0,0,7,0,1,0,0,1,0,1,0,1,0,1
3,1,1,35,1,0,53,0,0,0,0,1,0,0,0,1,0,1
4,0,3,35,0,0,8,1,1,1,0,1,0,1,1,0,0,1


In [50]:
y = titanic['survived']
X = titanic.drop(columns=['survived'])

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [62]:
classificador_base = DecisionTreeClassifier()

modelo_bagging = BaggingClassifier(classificador_base, n_estimators=50, random_state=42)

modelo_bagging.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=50,
                  random_state=42)

In [66]:
y_pred = modelo_bagging.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7359550561797753